<a href="https://colab.research.google.com/github/gunisha30/projects/blob/master/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from nltk.corpus import stopwords
df = pd.read_csv("//content//drive//My Drive//spam.csv")
#frequency of ham and spam 
pd.crosstab(df['Label'],columns='freq')
#stemming
ps = nltk.PorterStemmer()
#list of stopwords
stopwords = nltk.corpus.stopwords.words('english')
#returns percentage of punctuations in text
def punct_pc(text):
    punct_count = sum([1 for char in text if char in string.punctuation])
    return (punct_count/(len(text) - text.count(' ')))*100

df['text_length'] = df['EmailText'].apply(lambda x : len(x)-x.count(' '))
df['punct'] = df['EmailText'].apply(lambda x : punct_pc(x))

#remove punctuations
def clean_data(text):
    punct = "".join([word.lower() for word in text if word not in string.punctuation])
    splt = re.split('\W+',punct)
    txt = [ps.stem(word) for word in splt if word not in stopwords]
    return txt

x_train,x_test,y_train,y_test = train_test_split(df[['EmailText','text_length','punct']],df['Label'],test_size=0.2,random_state=123)
#y_train.head()

Tfidf_Vect = CountVectorizer(analyzer=clean_data)
Tfidf_vect_fit = Tfidf_Vect.fit(x_train['EmailText'])
x_train_Tfidf_vect = Tfidf_vect_fit.transform(x_train['EmailText'])
x_test_Tfidf_vect = Tfidf_vect_fit.transform(x_test['EmailText'])
x_train_vect = pd.concat([x_train[['text_length','punct']].reset_index(drop=True) ,
                         pd.DataFrame(x_train_Tfidf_vect.toarray())],axis=1)


x_test_vect = pd.concat([x_test[['text_length','punct']].reset_index(drop=True) , 
                        pd.DataFrame(x_test_Tfidf_vect.toarray())],axis=1)

rf = RandomForestClassifier(random_state=123,n_jobs=3)
param = {'n_estimators' : [10,25,50,100,300], 'max_depth' : [10, 20, 50,100, None],'max_features' : [10,50,'auto']}
grid = GridSearchCV(rf,param,cv=5,n_jobs=3)

In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True